# CS171 Problem set 1, question 2 (10 pts)

In working on this assignment, do not feel constrained by the order or arrangement of the notebook as supplied.
You may not import other libraries, but while developing, you may write code as you wish, insert cells, run things in other orders, etc.  **Please clean up your notebook before submitting and check to make sure it runs in order from a fresh kernel!**

### Your task: implement naive Bayes for the case of binary classification and categorical features

You may assume that the features take on three values: 0, 1, 2.  And, that the labels are either 0 or 1.

Below is an example notebook that loads the data, trains a model, and outputs its error rate
**except**, the critical parts of `trainnaivebayes` and `naivebayesclass` which need to be implemented.  ***Implement them!***

Supplied with this notebook are `trainspam.txt` and `testspam.txt` which are the training and testing datasets for this problem.  `toytrain.txt` and `toytest.txt` are also given; they are not necessary, but have the data for the example problem from class (which might help with debugging).

In [1]:
import numpy as np
# do not import anything else!

In [2]:
# function to load the data
def loadsparsedata(fn):
    
    fp = open(fn,"r")
    lines = fp.readlines()
    maxf = 0;
    for line in lines:
        for i in line.split()[1::2]:
            maxf = max(maxf,int(i))
    
    X = np.zeros((len(lines),maxf))
    Y = np.zeros((len(lines)))
    
    for i, line in enumerate(lines):
        values = line.split()
        Y[i] = int(values[0])
        for j,v in zip(values[1::2],values[2::2]):
            X[i,int(j)-1] = int(v)
    
    return X,Y

In [3]:
### THIS IS WHAT YOU NEED TO IMPLEMENT
def trainnaivebayes(X,Y):
    # X is a numpy array: m-by-n (m examples, n features)
    # Y is a numpy array: m-by-1 (m examples, 1 label)
    (m,n) = X.shape;
    nclass = int(Y.max())+1; # in this case, nclass should be 2
    nval = int(X.max())+1;  # in this case, nval should be 3
    priorp = np.zeros([nclass])
    condp = np.zeros([n,nval,nclass])
    
    ### YOUR CODE HERE!!!
    #define the function for counting times of an element occurred
    def countp(sarray,k,col):
        count = 0
        for i in sarray[:,col]:
            if i == k:
                count += 1
        return count
    
    #assign elements to priorp
    mY = (np.mat(Y)).T
    for i in range(nclass):
        priorp[i] = countp(mY,i,0)/m
        
    #combine X and Y, and divided to two matrixs by y = k
    mX = np.mat(X)
    mXY = np.c_[mX,mY]
    mX0 = np.ones((1,n+1))
    mX1 = np.zeros((1,n+1))
    for i in range(m):
        if mXY[i,n] == 1:
            mX1 = np.r_[mX1,mXY[i]]
        else:
            mX0 = np.r_[mX0,mXY[i]]
    mX0 = np.delete(mX0,0,axis = 0)
    mX1 = np.delete(mX1,0,axis = 0)
    
    #assign elements to condp
    for i in range(n):
        for j in range(nval):
            condp[i,j,0] = float(countp(mX0,j,i))/float(mX0.shape[0])
            condp[i,j,1] = float(countp(mX1,j,i))/float(mX1.shape[0])
        
    # priorp is of shape (2,) and element k represents p(y=k)
    # condp is of shape (num-features, 3, 2) and element (i,j,k) is p(x_i = j | y = k)
    return priorp, condp

In [4]:
# load training data and train model (as an example... feel free to modify)
(trainX,trainY) = loadsparsedata("spamtrain.txt")
model = trainnaivebayes(trainX,trainY)

In [5]:
### THIS IS THE OTHER PART YOU NEED TO IMPLEMENT
def naivebayesclass(X,model):
    # X is the testing data (m-by-n)
    # model is the pair (priorp,condp) from trainnaivebayes
    (priorp,condp) = model
    
    ### YOUR CODE HERE!!!
    (m,n) = X.shape;
    predy = np.zeros([m])
    temp0 = 1.0
    temp1 = 1.0
    for i in range(m):
        for j in range(n):
            temp0 *= condp[j,int(X[i,j]),0]
            temp1 *= condp[j,int(X[i,j]),1]
        temp0 *= priorp[0]
        temp1 *= priorp[1]
        if temp0 < temp1:
            predy[i] = 1
        else:
            predy[i] = 0
    
    # predy should be an m-by-1 numpy array of the predicated classes
    return predy

In [6]:
def errorrate(Y,predY):
    return np.sum(Y.squeeze()!=predY.squeeze()) / float(Y.size)

In [7]:
# load testing data and test accuracy (as an example... feel free to modify for your testing)
(testX,testY) = loadsparsedata("spamtest.txt")
predy = naivebayesclass(testX,model)
err = errorrate(testY,predy)
print(err)

0.16511127063890882


In [8]:
def checkexample(trainX,trainY,testX,ansY,ansmodel):
    model = trainnaivebayes(trainX,trainY)
    np.testing.assert_equal(ansmodel,model)
    predy = naivebayesclass(testX,model)
    np.testing.assert_equal(ansY,predy)

In [9]:
# Test on toy problem:
(toytrainX,toytrainY) = loadsparsedata("toytrain.txt")
(toytestX,toytestY) = loadsparsedata("toytest.txt")
toymodelans = (np.array([0.6,0.4]),
               np.array([[[5./6, 0.50],
                          [1./6, 0.50],
                          [0.  , 0.  ]],
                         [[1./6, 0.50],
                          [1./6, 0.25],
                          [4./6, 0.25]],
                         [[5./6, 0.25],
                          [1./6, 0.75],
                          [0.  , 0.  ]]]))
checkexample(toytrainX,toytrainY,toytestX,toytestY,toymodelans)